# baseline v3

이 베이스라인 코드는 `사전학습 모델 로드`, `배치 학습`, `파인튜닝`, `양자화`, `PEFT` 등이 적용된 버전입니다.

Colab의 GPU 환경에서 개발되었습니다.
- 런타임 - 런타임 유형 변경 - GPU로 변경(T4 GPU 등)



# 환경 준비

개발 환경에 필요한 라이브러리 버전을 고정하고 최신 버전으로 라이브러리를 업데이트합니다.

- 아래 셀 실행
- 실행 완료 후 런타임 - 세션 다시 시작

In [ ]:
!pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" datasets pillow pandas torch torchvision --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86

# 데이터 준비

개발에 필요한 데이터를 준비합니다.

- train.csv, train 폴더
- test.csv, test 폴더
- sample_submission.csv

본 베이스라인은 colab에서 구글 드라이브를 마운트하여 사용합니다.

데이터를 압축 해제하는데 몇 분 정도의 시간이 소요됩니다.

#### 실습 참고 내용

    챕터 2-2 합성 데이터 실습
    - 구글 드라이브 마운트 : drive()

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 압축 해제
!unzip "/content/drive/My Drive/250918/data.zip" -d "/content/"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/test/test_2777.jpg  
  inflating: /content/test/test_2778.jpg  
  inflating: /content/test/test_2779.jpg  
  inflating: /content/test/test_2780.jpg  
  inflating: /content/test/test_2781.jpg  
  inflating: /content/test/test_2782.jpg  
  inflating: /content/test/test_2783.jpg  
  inflating: /content/test/test_2784.jpg  
  inflating: /content/test/test_2785.jpg  
  inflating: /content/test/test_2786.jpg  
  inflating: /content/test/test_2787.jpg  
  inflating: /content/test/test_2788.jpg  
  inflating: /content/test/test_2789.jpg  
  inflating: /content/test/test_2790.jpg  
  inflating: /content/test/test_2791.jpg  
  inflating: /content/test/test_2792.jpg  
  inflating: /content/test/test_2793.jpg  
  inflating: /content/test/test_2794.jpg  
  inflating: /content/test/test_2795.jpg  
  inflating: /content/test/test_2796.jpg  
  inflating: /content/test/test_2797.jpg  
  inflating: /content/test/test_2798.jpg  
  inflating: /cont

# 라이브러리, 데이터, 설정

In [26]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForImageTextToText,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
MODEL_ID = "../Qwen3-VL-8B-Instruct_Local"
IMAGE_SIZE = 384
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")


Device: cuda


# 모델, Processor

7.5GB 정도의 모델 다운로드가 진행됩니다. 10~20분 정도가 소요됩니다.

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - LoRA 구현 : LoraConfig()

In [27]:
# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

/home/team250/.local/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2291: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 21,823,488 || all params: 8,788,947,184 || trainable%: 0.2483


# 프롬프트 템플릿

#### 실습 참고 내용

    챕터 5-1 PEFT(파라미터 효율적 튜닝)
    - 프롬프트 템플릿 : convert_to_chatml(), formatting_prompts_func()

In [ ]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are an intelligent assistant specialized in visual question answering. "
    "You will receive a question about an image along with four possible answers. "
    "Select the correct answer using only one of the following letters: a, b, c, or d. "
    "Do not provide any explanation or additional information—just output the correct letter."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"Question: {question}\n\n"
        f"Options:\n"
        f"(a) {a}\n"
        f"(b) {b}\n"
        f"(c) {c}\n"
        f"(d) {d}\n\n"
        "Choose one letter (a, b, c, or d) that corresponds to the correct answer."
    )

# Custom Dataset, Collator

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - TensorDataset()

    챕터 5-2 데이터 생성 및 파인튜닝 (향후 학습 분량)
    - IntentDataset()

In [25]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages, "image": img}

# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        if self.train:
            enc["labels"] = enc["input_ids"].clone()

        return enc


# DataLoader

#### 실습 참고 내용

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 데이터로더 정의 : DataLoader()

In [ ]:
# # # 검증용 데이터 분리
# split = int(len(train_df)*0.8)
# train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

valid_subset = train_df.sample(n=600, random_state=999).reset_index(drop=True)
train_df = train_df.drop(valid_subset.index).reset_index(drop=True)

# # VQAMCDataset 형태로 변환
# train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# # 데이터로더
# train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

# fine-tuning

- 200개만 학습 : 10~20분 소요

#### 실습 참고 내용

    챕터 1-2 MLP 구현
    - 모델 정의 : SimpleMLP(), SequentialMLP()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [ ]:
from tqdm.auto import tqdm

model = model.to(device)
GRAD_ACCUM = 4

# 옵티마이저, 학습 스케줄러
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# num_training_steps = 1 * math.ceil(len(train_loader)/GRAD_ACCUM)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps*0.03), num_training_steps)

# 스케일러
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 학습 루프
global_step = 0
for epoch in range(8):
    train_subset = train_df.sample(n=500, random_state=SEED + epoch).reset_index(drop=True)
    train_ds = VQAMCDataset(train_subset, processor, train=True)
    train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)

    num_training_steps = 1 * math.ceil(len(train_loader)/GRAD_ACCUM)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps*0.03), num_training_steps)
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    for step, batch in enumerate(progress_bar, start=1):
        batch = {k:v.to(device) for k,v in batch.items()}
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            loss = outputs.loss / GRAD_ACCUM

        scaler.scale(loss).backward()
        running += loss.item()

        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0

    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()



/tmp/ipykernel_15068/756744154.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Epoch 1 [train]:   0%|          | 0/500 [00:00<?, ?batch/s]

/tmp/ipykernel_15068/756744154.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipykernel_15068/756744154.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):


Epoch 1 [valid]:   0%|          | 0/100 [00:00<?, ?batch/s]

[Epoch 1] valid loss 3.8598


...

[Epoch 7] valid loss 3.7726


Epoch 8 [train]:   0%|          | 0/500 [00:00<?, ?batch/s]

Epoch 8 [valid]:   0%|          | 0/100 [00:00<?, ?batch/s]

[Epoch 8] valid loss 3.7697


PermissionError: [Errno 13] Permission denied: '/content'

In [29]:
# 모델 저장
SAVE_DIR = "qwen3_vl_8B_lora_output"
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved:", SAVE_DIR)


Saved: qwen3_vl_8B_lora_output


# inference

30분~1시간 소요

#### 실습 참고 내용

    챕터4-1 RAG 기반 Customer Service AI 에이전트 개발
    - 데이터 파서 : langchain_core.output_parsers(), StrOutputParser()

    챕터 3-1 Transfer Learning 기반의 CNN 모델 학습
    - 학습 루프 : 문제 6: 모델 학습을 위한 반복문
    - 추론 : with torch.no_grad(), model.eval()

In [23]:
# 데이터 파서 : 모델의 응답에서 선지를 추출
def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"

# 추론을 위해 모든 레이어 활성화
model.eval()
preds = []

# 추론 루프
for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {"role":"user","content":[
            {"type":"image","image":img},
            {"type":"text","text":user_text}
        ]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False,
                                 eos_token_id=processor.tokenizer.eos_token_id)
    output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    # print("output_text:", output_text)
    # print("extract_choice:", extract_choice(output_text))
    preds.append(extract_choice(output_text))

# 제출 파일 생성
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})

Inference:   0%|          | 0/3887 [00:00<?, ?sample/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [24]:
submission.to_csv("submission_final.csv", index=False)

In [ ]:
# 모델 응답 예시
print(output_text)

system
You are a helpful visual question answering assistant. Answer using exactly one letter among a, b, c, or d. No explanation.
user
이 사진의 주요 상황은 무엇인가요?
(a) 수업 시간에 공부하고 있다
(b) 회의에 참석하고 있다
(c) 졸업식 준비 중이다
(d) 시험을 치르고 있다

정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.
assistant
c


In [ ]:
=== continous Fine-tuning =====

In [1]:
import os
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from peft import PeftModel

# 1. 원본 모델과 PEFT 모델 로드 경로는 이미 설정되어 있다고 가정합니다.
MODEL_ID = "../Qwen3-VL-8B-Instruct_Local"
LORA_DIR = "qwen3_vl_8B_lora_output" # 학습 결과물이 저장된 경로

# 2. 베이스 모델을 다시 로드합니다 (이미 로드되어 있지 않다면).
# *주의: 4bit/8bit 양자화로 로드했다면, 병합 전에 베이스 모델을 Full Precision으로 로드하는 것이 권장됩니다.
# 메모리 절약을 위해 양자화 상태에서 병합하고 저장할 수도 있습니다.
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,  # 합칠 때는 bfloat16 또는 float16 권장
    device_map="auto"
)

# 3. LoRA 가중치를 적용합니다.
model_to_merge = PeftModel.from_pretrained(base_model, LORA_DIR)

# 4. LoRA 가중치를 베이스 모델에 병합합니다.
# merge_and_unload()를 호출하면 메모리에서 LoRA 관련 모듈이 해제됩니다.
merged_model = model_to_merge.merge_and_unload()

# 5. 새로운 통합 모델을 저장할 디렉토리 설정
NEW_PRETRAINED_DIR = "qwen3_vl_8b_merged_v2"
os.makedirs(NEW_PRETRAINED_DIR, exist_ok=True)

# 6. 병합된 모델과 프로세서를 새로운 디렉토리에 저장합니다.
merged_model.save_pretrained(NEW_PRETRAINED_DIR, safe_serialization=True)
# 프로세서는 기존 LoRA 폴더에 저장된 것을 사용해도 되지만, 명확성을 위해 다시 저장합니다.
processor = AutoProcessor.from_pretrained(MODEL_ID) 
processor.save_pretrained(NEW_PRETRAINED_DIR)

print(f"새로운 사전 학습 모델이 생성되었습니다: {NEW_PRETRAINED_DIR}")

/home/team250/.local/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2291: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

새로운 사전 학습 모델이 생성되었습니다: qwen3_vl_8b_merged_v2


In [2]:
NEW_PRETRAINED_DIR = "qwen3_vl_8b_merged_v2" 

# MODEL_ID를 새로운 통합 모델의 경로로 설정
MODEL_ID = NEW_PRETRAINED_DIR

In [ ]:
#===== 1번 넘버 시드 ============

In [3]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Dict, List, Any
from transformers import (
    AutoModelForImageTextToText,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 사전 학습 모델 정의
IMAGE_SIZE = 384
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")


Device: cuda


In [4]:
# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 프로세서
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

# 사전학습 모델
base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# 양자화 모델로 로드
base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

# LoRA 세팅
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)

# PEFT 모델 생성
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

/home/team250/.local/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2291: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 21,823,488 || all params: 8,788,947,184 || trainable%: 0.2483


In [5]:
# 모델 지시사항
SYSTEM_INSTRUCT = (
    "You are an intelligent assistant specialized in visual question answering. "
    "You will receive a question about an image along with four possible answers. "
    "Select the correct answer using only one of the following letters: a, b, c, or d. "
    "Do not provide any explanation or additional information—just output the correct letter."
)

# 프롬프트
def build_mc_prompt(question, a, b, c, d):
    return (
        f"Question: {question}\n\n"
        f"Options:\n"
        f"(a) {a}\n"
        f"(b) {b}\n"
        f"(c) {c}\n"
        f"(d) {d}\n\n"
        "Choose one letter (a, b, c, or d) that corresponds to the correct answer."
    )

In [6]:
# 커스텀 데이터셋
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"])
        user_text = build_mc_prompt(q, a, b, c, d)

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role":"assistant","content":[{"type":"text","text":gold}]})

        return {"messages": messages, "image": img}

# 데이터 콜레이터
@dataclass
class DataCollator:
    processor: Any
    train: bool = True

    def __call__(self, batch):
        texts, images = [], []
        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        if self.train:
            enc["labels"] = enc["input_ids"].clone()

        return enc


In [ ]:
# =========== 1번 넘버 시드 ===========

In [7]:
# 학습데이터 200개만 추출
train_df = train_df.sample(n=200, random_state=1).reset_index(drop=True)

In [8]:
# 검증용 데이터 분리
split = int(len(train_df)*0.8)
train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

In [9]:
from tqdm.auto import tqdm
import math
import torch
from transformers import get_linear_schedule_with_warmup # 이미 import되어 있다고 가정

# --- 설정 변수 ---
# 학습할 총 에폭(Epoch) 횟수를 설정합니다.
EPOCHS = 3  # 기본 1회에서 3회로 늘려서 반복 학습을 수행하도록 수정
GRAD_ACCUM = 4

# 모델을 장치로 이동 (이미 되어 있음)
model = model.to(device)

# 옵티마이저, 학습 스케줄러 설정 (유지)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 학습 스케줄러 계산: (EPOCHS * 총 배치 수 / 기울기 누적 횟수)
num_training_steps = EPOCHS * math.ceil(len(train_loader) / GRAD_ACCUM) 
scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.03), num_training_steps)

# 스케일러 (AMP 사용을 위해 유지)
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 학습 루프
global_step = 0
for epoch in range(EPOCHS): # <-- 학습 반복 (Epochs)
    
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    
    # train_loader에서 미니배치(batch)를 하나씩 가져와 처리
    for step, batch in enumerate(progress_bar, start=1): # <-- 미니배치 처리 반복 (Steps)
        # 1. 미니배치 데이터를 GPU(device)로 이동
        batch = {k:v.to(device) for k,v in batch.items()}
        
        # 2. 순전파 및 손실 계산 (AMP 적용)
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            # 손실을 GRAD_ACCUM 횟수만큼 나누어 누적 (기울기 누적 적용)
            loss = outputs.loss / GRAD_ACCUM 

        # 3. 역전파: 누적된 손실로 기울기 계산
        scaler.scale(loss).backward()
        running += loss.item()

        # 4. 옵티마이저 스텝: GRAD_ACCUM 횟수만큼 누적되면 가중치 업데이트
        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True) # 기울기 초기화
            scheduler.step() # 학습률 업데이트
            global_step += 1

            # 로그 출력
            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0
        
        # 마지막 미니배치에서 누적 횟수가 채워지지 않아도 업데이트하도록 처리 (선택 사항)
        if step == len(train_loader) and step % GRAD_ACCUM != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1
            avg_loss = running / (step % GRAD_ACCUM)
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})


    # 5. 검증 루프 (Validation Loop) - 에폭 종료 후 수행 (유지)
    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()

/tmp/ipykernel_15890/57108990.py:21: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Epoch 1 [train]:   0%|          | 0/160 [00:00<?, ?batch/s]

/tmp/ipykernel_15890/57108990.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipykernel_15890/57108990.py:73: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):


Epoch 1 [valid]:   0%|          | 0/40 [00:00<?, ?batch/s]

[Epoch 1] valid loss 3.7805


...

Epoch 2 [valid]:   0%|          | 0/40 [00:00<?, ?batch/s]

[Epoch 2] valid loss 3.7815


Epoch 3 [train]:   0%|          | 0/160 [00:00<?, ?batch/s]

Epoch 3 [valid]:   0%|          | 0/40 [00:00<?, ?batch/s]

[Epoch 3] valid loss 3.7828


In [ ]:
# =========== 2번 넘버 시드 ===========

In [20]:
# 학습데이터 200개만 추출
train_df = train_df.sample(n=200, random_state=10).reset_index(drop=True)

# 검증용 데이터 분리
split = int(len(train_df)*1)
train_subset, valid_subset = train_df.iloc[:], train_df.iloc[-1]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

In [21]:
from tqdm.auto import tqdm
import math
import torch
from transformers import get_linear_schedule_with_warmup # 이미 import되어 있다고 가정

# --- 설정 변수 ---
# 학습할 총 에폭(Epoch) 횟수를 설정합니다.
EPOCHS = 3  # 기본 1회에서 3회로 늘려서 반복 학습을 수행하도록 수정
GRAD_ACCUM = 4

# 모델을 장치로 이동 (이미 되어 있음)
model = model.to(device)

# 옵티마이저, 학습 스케줄러 설정 (유지)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 학습 스케줄러 계산: (EPOCHS * 총 배치 수 / 기울기 누적 횟수)
num_training_steps = EPOCHS * math.ceil(len(train_loader) / GRAD_ACCUM) 
scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.03), num_training_steps)

# 스케일러 (AMP 사용을 위해 유지)
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 학습 루프
global_step = 0
for epoch in range(EPOCHS): # <-- 학습 반복 (Epochs)
    
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    
    # train_loader에서 미니배치(batch)를 하나씩 가져와 처리
    for step, batch in enumerate(progress_bar, start=1): # <-- 미니배치 처리 반복 (Steps)
        # 1. 미니배치 데이터를 GPU(device)로 이동
        batch = {k:v.to(device) for k,v in batch.items()}
        
        # 2. 순전파 및 손실 계산 (AMP 적용)
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            # 손실을 GRAD_ACCUM 횟수만큼 나누어 누적 (기울기 누적 적용)
            loss = outputs.loss / GRAD_ACCUM 

        # 3. 역전파: 누적된 손실로 기울기 계산
        scaler.scale(loss).backward()
        running += loss.item()

        # 4. 옵티마이저 스텝: GRAD_ACCUM 횟수만큼 누적되면 가중치 업데이트
        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True) # 기울기 초기화
            scheduler.step() # 학습률 업데이트
            global_step += 1

            # 로그 출력
            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0
        
        # 마지막 미니배치에서 누적 횟수가 채워지지 않아도 업데이트하도록 처리 (선택 사항)
        if step == len(train_loader) and step % GRAD_ACCUM != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1
            avg_loss = running / (step % GRAD_ACCUM)
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})


    # 5. 검증 루프 (Validation Loop) - 에폭 종료 후 수행 (유지)
    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()

/tmp/ipykernel_15890/57108990.py:21: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Epoch 1 [train]:   0%|          | 0/200 [00:00<?, ?batch/s]

/tmp/ipykernel_15890/57108990.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


KeyboardInterrupt: 

In [ ]:
# =========== 3번 넘버 시드 ===========

In [ ]:
# 학습데이터 200개만 추출
train_df = train_df.sample(n=200, random_state=3).reset_index(drop=True)

# 검증용 데이터 분리
split = int(len(train_df)*1)
train_subset, valid_subset = train_df.iloc[:], train_df.iloc[-1]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

In [ ]:
from tqdm.auto import tqdm
import math
import torch
from transformers import get_linear_schedule_with_warmup # 이미 import되어 있다고 가정

# --- 설정 변수 ---
# 학습할 총 에폭(Epoch) 횟수를 설정합니다.
EPOCHS = 3  # 기본 1회에서 3회로 늘려서 반복 학습을 수행하도록 수정
GRAD_ACCUM = 4

# 모델을 장치로 이동 (이미 되어 있음)
model = model.to(device)

# 옵티마이저, 학습 스케줄러 설정 (유지)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 학습 스케줄러 계산: (EPOCHS * 총 배치 수 / 기울기 누적 횟수)
num_training_steps = EPOCHS * math.ceil(len(train_loader) / GRAD_ACCUM) 
scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.03), num_training_steps)

# 스케일러 (AMP 사용을 위해 유지)
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 학습 루프
global_step = 0
for epoch in range(EPOCHS): # <-- 학습 반복 (Epochs)
    
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    
    # train_loader에서 미니배치(batch)를 하나씩 가져와 처리
    for step, batch in enumerate(progress_bar, start=1): # <-- 미니배치 처리 반복 (Steps)
        # 1. 미니배치 데이터를 GPU(device)로 이동
        batch = {k:v.to(device) for k,v in batch.items()}
        
        # 2. 순전파 및 손실 계산 (AMP 적용)
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            # 손실을 GRAD_ACCUM 횟수만큼 나누어 누적 (기울기 누적 적용)
            loss = outputs.loss / GRAD_ACCUM 

        # 3. 역전파: 누적된 손실로 기울기 계산
        scaler.scale(loss).backward()
        running += loss.item()

        # 4. 옵티마이저 스텝: GRAD_ACCUM 횟수만큼 누적되면 가중치 업데이트
        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True) # 기울기 초기화
            scheduler.step() # 학습률 업데이트
            global_step += 1

            # 로그 출력
            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0
        
        # 마지막 미니배치에서 누적 횟수가 채워지지 않아도 업데이트하도록 처리 (선택 사항)
        if step == len(train_loader) and step % GRAD_ACCUM != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1
            avg_loss = running / (step % GRAD_ACCUM)
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})


    # 5. 검증 루프 (Validation Loop) - 에폭 종료 후 수행 (유지)
    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()

In [ ]:
# =========== 4번 넘버 시드 ===========

In [ ]:
# 학습데이터 200개만 추출
train_df = train_df.sample(n=200, random_state=4).reset_index(drop=True)

# 검증용 데이터 분리
split = int(len(train_df)*1)
train_subset, valid_subset = train_df.iloc[:], train_df.iloc[-1]

# VQAMCDataset 형태로 변환
train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

# 데이터로더
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True), num_workers=0)
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True), num_workers=0)

In [ ]:
from tqdm.auto import tqdm
import math
import torch
from transformers import get_linear_schedule_with_warmup # 이미 import되어 있다고 가정

# --- 설정 변수 ---
# 학습할 총 에폭(Epoch) 횟수를 설정합니다.
EPOCHS = 3  # 기본 1회에서 3회로 늘려서 반복 학습을 수행하도록 수정
GRAD_ACCUM = 4

# 모델을 장치로 이동 (이미 되어 있음)
model = model.to(device)

# 옵티마이저, 학습 스케줄러 설정 (유지)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 학습 스케줄러 계산: (EPOCHS * 총 배치 수 / 기울기 누적 횟수)
num_training_steps = EPOCHS * math.ceil(len(train_loader) / GRAD_ACCUM) 
scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.03), num_training_steps)

# 스케일러 (AMP 사용을 위해 유지)
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 학습 루프
global_step = 0
for epoch in range(EPOCHS): # <-- 학습 반복 (Epochs)
    
    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    
    # train_loader에서 미니배치(batch)를 하나씩 가져와 처리
    for step, batch in enumerate(progress_bar, start=1): # <-- 미니배치 처리 반복 (Steps)
        # 1. 미니배치 데이터를 GPU(device)로 이동
        batch = {k:v.to(device) for k,v in batch.items()}
        
        # 2. 순전파 및 손실 계산 (AMP 적용)
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            # 손실을 GRAD_ACCUM 횟수만큼 나누어 누적 (기울기 누적 적용)
            loss = outputs.loss / GRAD_ACCUM 

        # 3. 역전파: 누적된 손실로 기울기 계산
        scaler.scale(loss).backward()
        running += loss.item()

        # 4. 옵티마이저 스텝: GRAD_ACCUM 횟수만큼 누적되면 가중치 업데이트
        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True) # 기울기 초기화
            scheduler.step() # 학습률 업데이트
            global_step += 1

            # 로그 출력
            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0
        
        # 마지막 미니배치에서 누적 횟수가 채워지지 않아도 업데이트하도록 처리 (선택 사항)
        if step == len(train_loader) and step % GRAD_ACCUM != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1
            avg_loss = running / (step % GRAD_ACCUM)
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})


    # 5. 검증 루프 (Validation Loop) - 에폭 종료 후 수행 (유지)
    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vb in tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"):
            vb = {k:v.to(device) for k,v in vb.items()}
            val_loss += model(**vb).loss.item()
            val_steps += 1
    print(f"[Epoch {epoch+1}] valid loss {val_loss/val_steps:.4f}")
    model.train()

In [ ]:
# =========== 5번 넘버 시드 ===========

In [22]:
# 모델 저장
SAVE_DIR = "qwen3_vl_8B_lora_output_final"
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved:", SAVE_DIR)


Saved: qwen3_vl_8B_lora_output_final


In [ ]:
submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv("submission3.csv", index=False)